In [ ]:
import keras
from keras_nlp.layers import PositionEmbedding
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import layers
import numpy as np
import pandas as pd


In [14]:

# Your input data
data = pd.read_csv(
    "../data/LJSpeech-1.1/metadata.csv",
    sep="|",
    header=None,
    names=["ID", "Text1", "Text2"],
)
texts = data["Text1"].to_list()
ID = data["ID"].to_list()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
num_classes = len(tokenizer.word_index) + 1  # Add 1 for the padding token
sequences = tokenizer.texts_to_sequences(texts)
X_data = pad_sequences(sequences, padding="post", maxlen=30)
print(num_classes)

# Reshape to (13100 * 30, 1)
reshaped_input = X_data.reshape((-1, 1))

# Positional embedding layer
class PositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, **kwargs):
        super(PositionEmbedding, self).__init__(**kwargs)
        self.sequence_length = sequence_length

    def build(self, input_shape):
        feature_length = input_shape[-1]
        self.position_embeddings = self.add_weight(
            shape=(self.sequence_length, feature_length),
            initializer=tf.keras.initializers.RandomNormal(),
            trainable=True,
        )

    def call(self, inputs):
        start_index = 0
        sequence_length = tf.shape(self.position_embeddings)[0]
        feature_length = tf.shape(self.position_embeddings)[-1]

        position_embeddings = tf.tile(
            tf.slice(
                self.position_embeddings,
                (start_index, 0),
                (sequence_length, feature_length),
            ),
            [tf.shape(inputs)[0] // sequence_length + 1, 1],
        )

        return tf.slice(position_embeddings, (0, 0), tf.shape(inputs))

# Example usage
position_embedding_layer = PositionEmbedding(sequence_length=30)
embedded_output = position_embedding_layer(reshaped_input)
embedded_output = embedded_output.numpy()  # Convert to NumPy array for printing
print(embedded_output.shape)
embedded_output = embedded_output.reshape((13100, 30, -1))
embedded_output[0]


14518
(393000, 1)


array([[-0.01198915],
       [-0.0633029 ],
       [ 0.0093736 ],
       [ 0.04419341],
       [-0.01661698],
       [-0.05808525],
       [-0.00037932],
       [-0.00964375],
       [ 0.02145249],
       [-0.03655505],
       [-0.09183893],
       [-0.02409197],
       [-0.08685187],
       [ 0.02324911],
       [ 0.06045104],
       [-0.03361482],
       [-0.08698981],
       [ 0.0050036 ],
       [ 0.0931929 ],
       [-0.05324392],
       [-0.03192737],
       [-0.0963658 ],
       [-0.07770997],
       [-0.00902212],
       [ 0.02158431],
       [-0.03013596],
       [-0.05272006],
       [-0.08228543],
       [ 0.02043069],
       [-0.07138651]], dtype=float32)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionEmbedding(layers.Layer):
    def __init__(self, sequence_length, **kwargs):
        super(PositionEmbedding, self).__init__(**kwargs)
        self.sequence_length = sequence_length

    def build(self, input_shape):
        feature_dim = input_shape[-1]
        self.position_embeddings = self.add_weight(
            shape=(self.sequence_length, feature_dim),
            initializer="uniform",
            trainable=True,
        )

    def call(self, inputs):
        start_index = 0
        sequence_length = self.sequence_length
        feature_length = tf.shape(self.position_embeddings)[-1]

        position_embeddings = tf.slice(
            self.position_embeddings,
            (start_index, 0),
            (sequence_length, feature_length),
        )
        return tf.broadcast_to(position_embeddings, tf.shape(inputs))

# Your input data
data = pd.read_csv(
    "../data/LJSpeech-1.1/metadata.csv",
    sep="|",
    header=None,
    names=["ID", "Text1", "Text2"],
)
texts = data["Text1"].to_list()
ID = data["ID"].to_list()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
num_classes = len(tokenizer.word_index) + 1  # Add 1 for the padding token
sequences = tokenizer.texts_to_sequences(texts)
X_data = pad_sequences(sequences, padding="post", maxlen=30)

# Reshape to (13100 * 30, 1)
reshaped_input = X_data.reshape((-1, 1))

# Positional embedding layer
position_embedding_layer = PositionEmbedding(sequence_length=30)

# Apply positional embedding
embedded_output = position_embedding_layer(reshaped_input)

# Reshape back to (13100, 30, embedding_dim)
embedded_output = embedded_output.numpy().reshape((13100, 30, -1))

# Display the shape of the output
print(embedded_output.shape)


In [ ]:
import numpy as np

# Example input data
batch_size = 13100
sequence_length = 30

# Create a sample input tensor with sequential positions
input_positions = np.tile(np.arange(sequence_length), (batch_size, 1))

# Display the shape of the input tensor
print(input_positions.shape)